In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import json
import sys
from PIL import Image
import requests
from io import BytesIO
import os
import random
import numpy as np

In [2]:
goodreads_parent = "https://www.goodreads.com"

# Retrieving relevant values

In [10]:
with open('data_2.json', 'r') as f:
    data = json.load(f)

book_links = data['book_links']
shelf_links = data['shelves']
shelf_values = data['shelf_values']
img_urls = data['img_urls']
len(shelf_values)

10000

# Prettified Scraping

### Run the following code to scrape the relevant data from the GoodReads website

#### Book Links

In [ ]:
book_links = []

book_links = []
for i in range(100):
    url = f"https://www.goodreads.com/list/show/1.Best_Books_Ever?page={i + 1}"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    for book_item in soup.find_all("table")[0].find_all("tr"):
        href = None
        for a_item in book_item.find_all("a"):
            if a_item.get("href") is not None and a_item.get("title") is not None:
                href = a_item["href"]
                break
        book_links.append(href)

print(len(book_links))

#### Shelf Links

In [ ]:
shelf_links = []

count = 0
for book_link in book_links:
    count += 1
    try:
        if count > len(shelf_links):
            sys.stdout.write("\r" + str(len(shelf_links)) + "/" + str(len(book_links)))
            sys.stdout.flush()
            url = goodreads_parent + book_link
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            r = re.search(
                r'"shelvesUrl":"(.*)","pub',
                soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0]
            )
            shelf_links.append(r.group(1))s
    except:
        shelf_links.append("Error")

        
print(len(shelf_links))

#### Shelf Values

In [ ]:
shelf_values = []

errors = 0
updates = 0
for i in range(len(shelf_links)):
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(shelf_links)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        if shelf_values[i] == "Error" or (isinstance(shelf_values[i], str) \
                                          and shelf_values[i].startswith("Error")) \
        or (shelf_values[i].count("to-read") > 1 and shelf_values[i].count("to-read") == len(shelf_values[i])):
            tmp = []
            for j in range(10):
                page = urlopen(shelf_links[i] + f"?page={j}")
                html = page.read().decode("utf-8")
                soup = BeautifulSoup(html, "html.parser")
                divs = soup.find_all("div", {"class": "shelfStat"})
                for div in divs:
                    tmp.append(div.findChildren("a")[0].text)
                if len(divs) < 100:
                    break
            shelf_values[i] = tmp
            tmp_shelf_data = {
                "shelves": shelf_links,
                "values": shelf_values
            }
            updates += 1
    except Exception as e:
        shelf_values[i] = "Error " + str(e)
        errors += 1
    if i % 100 == 0 and updates > 0:
        with open('tmp_shelf_data.json', 'w', encoding='utf-8') as f:
            json.dump(tmp_shelf_data, f, ensure_ascii=False, indent=4)

#### Book Covers

In [19]:
# GETTING URLS
if not 'img_urls' in locals():
    img_urls = []

errors = 0

for i in range(len(book_links)):
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(book_links)) + " errors: " + str(errors))
    sys.stdout.flush()
    request_count = 0
    completed = False
    if not i < len(img_urls) or img_urls[i] == "Error":
        while request_count < 10 and not completed:
            try:
                page = urlopen(goodreads_parent + book_links[i])
                html = page.read().decode("utf-8")
                soup = BeautifulSoup(html, "html.parser")

                url = soup.find_all("div", {"class", "BookCover__image"})[0].findChildren("img")[0]["src"]

                if i < len(img_urls):
                    img_urls[i] = url
                else:
                    img_urls.append(url)
                
                completed = True
            except:
                request_count += 1
        
        if request_count >= 10:
            if i < len(img_urls):
                img_urls[i] = "Error"
            else:
                img_urls.append("Error")
            errors += 1

10000/10000 errors: 0

In [36]:
# Downloading files
directory = "./data/images/"
error_vals = []
errors = 0

for i in range(len(img_urls)):
    file_name = str(i) + ".jpg"
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(img_urls)) + " errors: " + str(errors))
    sys.stdout.flush()
    if not os.path.isfile(directory + file_name) or os.stat(directory + file_name).st_size == 0:      
        try:
            response = requests.get(img_urls[i])
            img = Image.open(BytesIO(response.content)).convert('RGB')
            img.save(directory + file_name)
        except Exception as e:
            error_vals.append("Error " + str(e) + " " + str(i) + " " + img_urls[i] + " " + goodreads_parent + book_links[i])
            errors += 1

10000/10000 errors: 4

In [37]:
error_vals

['Error cannot identify image file <_io.BytesIO object at 0x7fd782593090> 1120 https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1166542346i/13103.jpg https://www.goodreads.com/book/show/13103.The_Celestine_Prophecy',
 'Error cannot identify image file <_io.BytesIO object at 0x7fd7825933b0> 1388 https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1327880523i/13312527.jpg https://www.goodreads.com/book/show/13312527-the-opportunist',
 'Error cannot identify image file <_io.BytesIO object at 0x7fd78809ab30> 3099 https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1168447985i/33514.jpg https://www.goodreads.com/book/show/33514.The_Elements_of_Style',
 'Error cannot identify image file <_io.BytesIO object at 0x7fd78808dea0> 9073 https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1256068757i/2253484.jpg https://www.goodreads.com/book/show/2253484.Shadow_

In [38]:
shapes = set()
error_vals = []
errors = 0

for i in range(len(img_urls)):
    file_name = str(i) + ".jpg"
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(img_urls)) + " errors: " + str(errors))
    sys.stdout.flush()
    if os.path.isfile(directory + file_name):  
        try:
            img = Image.open(directory + file_name)
            img.load()
            data = np.asarray( img, dtype="int32" )
            shapes.add(data.shape)
        except:
            errors += 1
            error_vals.append(i)

10000/10000 errors: 0

#### Final Save

In [21]:
data = {}

data["shelf_values"] = shelf_values
data["book_links"] = book_links
data["img_urls"] = img_urls

with open('data_2.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# Initial Scraping

### Initial Loading

In [3]:
url = "https://www.goodreads.com/list/show/1.Best_Books_Ever"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [4]:
len(soup.find_all("table")[0].find_all("tr"))

100

In [5]:
soup.find_all("table")[0].find_all("tr")[0].find_all("a")[0]['href']

'/book/show/2767052-the-hunger-games'

In [73]:
book_links = []
for i in range(100):
    url = f"https://www.goodreads.com/list/show/1.Best_Books_Ever?page={i + 1}"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    for book_item in soup.find_all("table")[0].find_all("tr"):
        href = None
        for a_item in book_item.find_all("a"):
#             print("+++", a_item, a_item.get("href"))
            if a_item.get("href") is not None and a_item.get("title") is not None:
                href = a_item["href"]
                break
        book_links.append(href)
        if (len(book_links) == 20):
            print(book_links)
print(len(book_links))

['/book/show/2767052-the-hunger-games', '/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix', '/book/show/1885.Pride_and_Prejudice', '/book/show/2657.To_Kill_a_Mockingbird', '/book/show/19063.The_Book_Thief', '/book/show/41865.Twilight', '/book/show/170448.Animal_Farm', '/book/show/11127.The_Chronicles_of_Narnia', '/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set', '/book/show/11870085-the-fault-in-our-stars', '/book/show/18405.Gone_with_the_Wind', '/book/show/370493.The_Giving_Tree', '/book/show/6185.Wuthering_Heights', '/book/show/5297.The_Picture_of_Dorian_Gray', '/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone', '/book/show/10210.Jane_Eyre', '/book/show/22628.The_Perks_of_Being_a_Wallflower', '/book/show/968.The_Da_Vinci_Code', '/book/show/4671.The_Great_Gatsby', '/book/show/929.Memoirs_of_a_Geisha']
10000


In [13]:
url = "https://www.goodreads.com/book/show/2767052-the-hunger-games"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [25]:
# print(soup.prettify())

In [24]:
len(soup.findAll("script", {"id": "__NEXT_DATA__"}))

1

In [55]:
len(soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents)
# soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0]

1

In [57]:
json_data = json.loads(soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0])
r = re.search(r'"shelvesUrl":"(.*)","pub', soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0])
r.group(1)
# json_data['props']['pageProps']['apolloState']['Work:kca://work/amzn1.gr.work.v1.qDM8aVuWTQKRymQBUS02og']

'https://www.goodreads.com/work/shelves/2792775-the-hunger-games'

In [14]:
print(soup.findAll(text="/shelves/2792775-the-hunger-games"))

[]


In [28]:
print(soup.findAll("div", {"class": "Button__labelItem"}))

[]


In [60]:
# driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
# driver.get(url)
# text_field = driver.find_element(By.LINK_TEXT, "...more")
# text_field.click()
# # text_field = driver.find_element(By.LINK_TEXT, "...show all")
# text = wait(driver, 10).until(lambda driver: not text_field.text == '..more' and text_field.text)
# text

In [75]:
# book_links

In [ ]:
shelf_links = []

In [85]:
count = 0
for book_link in book_links:
    count += 1
    try:
        if count > len(shelf_links):
            sys.stdout.write("\r" + str(len(shelf_links)) + "/" + str(len(book_links)))
            sys.stdout.flush()
            url = goodreads_parent + book_link
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            r = re.search(
                r'"shelvesUrl":"(.*)","pub',
                soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0]
            )
            shelf_links.append(r.group(1))s
    except:
        shelf_links.append("Error")
print(len(shelf_links))

9999/1000010000


In [86]:
print(len(shelf_links))

10000


In [88]:
sum([1 for shelf in shelf_links if shelf == "Error"])

45

In [87]:
new_data = {
    "books" : book_links,
    "shelves" : shelf_links
}

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

### Plugging Holes in the Data

In [3]:
with open('data.json', 'r') as f:
    data = json.load(f)
    
book_links = data["books"]
shelf_links = data["shelves"]

len(book_links), len(shelf_links)

(10000, 10000)

In [13]:
count = 0
for i in range(len(book_links)):
    if shelf_links[i] == "Error":
        count += 1
        print(count, book_links[i])
        try:
            url = goodreads_parent + book_links[i]
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            r = re.search(
                r'"shelvesUrl":"(.*)","pub',
                soup.findAll("script", {"id": "__NEXT_DATA__"})[0].contents[0]
            )
            shelf_links[i] = r.group(1)
        except Exception as e:
            print("   FAILURE for", book_links[i])
            print("   ", e)

1 /book/show/1844941.Kuyucakl_Yusuf


In [5]:
sum([1 for shelf in shelf_links if shelf == "Error"])

1

In [6]:
len(book_links), len(shelf_links)

(10000, 10000)

In [14]:
new_data = {
    "books" : book_links,
    "shelves" : shelf_links
}

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

### Getting Shelves

In [22]:
shelf_link = "https://www.goodreads.com/work/shelves/2792775-the-hunger-games"
page = urlopen(shelf_link)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
print(len(soup.find_all("div", {"class": "shelfStat"})))
soup.find_all("div", {"class": "shelfStat"})[0].findChildren("a")[0].text

100


'to-read'

In [24]:
shelf_values = []

In [25]:
errors = 0
count = 0

for shelf_link in shelf_links:
    count += 1
    sys.stdout.write("\r" + str(len(shelf_values)) + "/" + str(len(shelf_links)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        if count > len(shelf_values):
            tmp = []
            for i in range(10):
                page = urlopen(shelf_link + f"?page={i}")
                html = page.read().decode("utf-8")
                soup = BeautifulSoup(html, "html.parser")
                for div in soup.find_all("div", {"class": "shelfStat"}):
                    tmp.append(soup.find_all("div", {"class": "shelfStat"})[0].findChildren("a")[0].text)
            shelf_values.append(tmp)
    except:
        shelf_values.append("Error")
        errors += 1

9999/10000 errors: 7646

In [32]:
# TEMPORARY
tmp_shelf_data = {
    "shelves": shelf_links,
    "values": shelf_values
}

with open('tmp_shelf_data.json', 'w', encoding='utf-8') as f:
    json.dump(tmp_shelf_data, f, ensure_ascii=False, indent=4)

In [37]:
count = 0
errors = 0

In [39]:
completed = count - 1

In [58]:
len

10000

In [59]:
completed = 0

In [102]:
tmp = []
for j in range(10):
    page = urlopen(shelf_links[0] + f"?page={j}")
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    for div in soup.find_all("div", {"class": "shelfStat"}):
        tmp.append(div.findChildren("a")[0].text)
# tmp

In [195]:
tmp_link = shelf_links[book_links.index("/book/show/33560584-tiger-and-the-robot")]
page = urlopen(tmp_link + f"?page={2}")
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
divs = soup.find_all("div", {"class": "shelfStat"})
len(divs)

0

In [7]:
# count = 0
errors = 0
updates = 0
for i in range(len(shelf_values)):
#     count += 1
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(shelf_links)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        if shelf_values[i] == "Error" or (isinstance(shelf_values[i], str) \
                                          and shelf_values[i].startswith("Error")) \
        or (shelf_values[i].count("to-read") > 1 and shelf_values[i].count("to-read") == len(shelf_values[i])):
    #         print(count, book_links[i])
            tmp = []
            for j in range(10):
                page = urlopen(shelf_links[i] + f"?page={j}")
                html = page.read().decode("utf-8")
                soup = BeautifulSoup(html, "html.parser")
                divs = soup.find_all("div", {"class": "shelfStat"})
                for div in divs:
                    tmp.append(div.findChildren("a")[0].text)
                if len(divs) < 100:
                    break
            shelf_values[i] = tmp
            tmp_shelf_data = {
                "shelves": shelf_links,
                "values": shelf_values
            }
            updates += 1
    except Exception as e:
        shelf_values[i] = "Error " + str(e)
        errors += 1
#         print("   FAILURE for", shelf_links[i])
#         print("   ", e)
    if i % 100 == 0 and updates > 0:
        with open('tmp_shelf_data.json', 'w', encoding='utf-8') as f:
            json.dump(tmp_shelf_data, f, ensure_ascii=False, indent=4)

10000/10000 errors: 0

In [83]:
len(shelf_values)

10000

In [100]:
# shelf_values[-1]

In [45]:
data["shelf_values"] = shelf_values
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### Images

In [112]:
page = urlopen(goodreads_parent + book_links[0])
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

soup.find_all("div", {"class", "BookCover__image"})[0].findChildren("img")[0]["src"]

'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052.jpg'

In [113]:
img_urls = []

In [120]:
# GETTING URLS
errors = 0

for i in range(len(book_links)):
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(book_links)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        if not i < len(img_urls) or img_urls[i] == "Error":
            page = urlopen(goodreads_parent + book_links[0])
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")

            url = soup.find_all("div", {"class", "BookCover__image"})[0].findChildren("img")[0]["src"]
            
            if i < len(img_urls):
                img_urls[i] = url
            else:
                img_urls.append(url)
    except Exception as e:
        if i < len(img_urls):
            img_urls[i] = "Error"
        else:
            img_urls.append("Error")
        errors += 1

10000/10000 errors: 0

In [115]:
len(img_urls)

10000

In [121]:
sum([1 if "Error" in img_url else 0 for img_url in img_urls])

0

In [130]:
dimensions = {}

In [153]:
# Checking the image sizes
response = requests.get(img_urls[i])
img = Image.open(BytesIO(response.content))
img.size

errors = 0
error_vals = []

for i in range(len(img_urls)):
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(img_urls)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        response = requests.get(img_urls[i])
        img = Image.open(BytesIO(response.content))
        dim = str(img.size[0]) + "," + str(img.size[1])
        if dim in dimensions:
            dimensions[dim] = dimensions[dim] + 1
        else:
            dimensions[dim] = 1
    except Exception as e:
        error_vals.append("Error " + str(i))
        errors += 1

10000/10000 errors: 0

In [155]:
dimensions, len(error_vals)

({'314,475': 20361}, 0)

In [150]:
len(img_urls)

10000

In [157]:
# Downloading files
directory = "./data/images/"
error_vals = []
errors = 0

for i in range(len(img_urls)):
    file_name = str(i) + ".jpg"
    sys.stdout.write("\r" + str(i + 1) + "/" + str(len(img_urls)) + " errors: " + str(errors))
    sys.stdout.flush()
    try:
        response = requests.get(img_urls[i])
        img = Image.open(BytesIO(response.content))
        if not os.path.isfile(directory + file_name):
            img.save(directory + file_name)
    except Exception as e:
        error_vals.append("Error " + str(i))
        errors += 1

10000/10000 errors: 0

### Possible Labels

In [172]:
# for i in range(5):
#     print(shelf_values[random.randint(0,10000)])

In [8]:
label_counts = {}

for shelf in shelf_values:
    for shelf_value in shelf:
        if shelf_value in label_counts:
            label_counts[shelf_value] = label_counts[shelf_value] + 1
        else:
            label_counts[shelf_value] = 1

In [9]:
len(label_counts)

648474

In [10]:
shelf_values[0].count("to-read")

4

In [4]:
set([shelf_value.count("to-read") for shelf_value in shelf_values])
# Should be {0, 1, 2, 3, 4, 6, 8, 10}

{0, 1, 2, 3, 4, 6, 8, 10}

In [12]:
label_count_sort = list(sorted(((label_counts[label], label) for label in label_counts), reverse=True))
len(label_count_sort), label_count_sort[0]

(648474, (29063, 'currently-reading'))

In [13]:
label_count_sort[0:50]

[(29063, 'currently-reading'),
 (20666, 'fiction'),
 (20485, 'to-read'),
 (20013, 'favorites'),
 (18996, 'own'),
 (18881, 'owned'),
 (18766, 'books-i-own'),
 (18105, 'wishlist'),
 (17897, 'library'),
 (17765, 'owned-books'),
 (17748, 'kindle'),
 (17525, 'to-buy'),
 (17238, 'ebook'),
 (17017, '2020'),
 (17015, '2021'),
 (16849, '2022'),
 (16789, 'my-library'),
 (16582, '2019'),
 (16576, 'my-books'),
 (16417, '2018'),
 (16180, '2017'),
 (16004, 'favourites'),
 (15768, '2016'),
 (15273, '2015'),
 (15271, 'audiobook'),
 (15150, 'default'),
 (15084, 'ebooks'),
 (14889, 'dnf'),
 (14734, 'audiobooks'),
 (14660, 'tbr'),
 (14579, '2014'),
 (14127, 'i-own'),
 (14064, 'novels'),
 (14023, 'wish-list'),
 (13800, 'audio'),
 (13570, 'young-adult'),
 (13450, '2013'),
 (13246, 'fantasy'),
 (13185, 'books'),
 (13045, 'adult'),
 (13004, 'bookshelf'),
 (12645, 'contemporary'),
 (12599, 'did-not-finish'),
 (12476, 'physical-tbr'),
 (12254, 'abandoned'),
 (12124, 'e-book'),
 (12084, '2023'),
 (12078, 'have'

In [16]:
lengths = list(sorted([len(shelf_value) for shelf_value in shelf_values]))
length_dict = {
    '0-999': 0,
    '1000-1999': 0,
    '2000-2999': 0,
    '3000-3999': 0,
    '4000-4999': 0,
    '5000-5999': 0,
    '6000-6999': 0,
    '7000-7999': 0,
    '8000-8999': 0,
    '9000-10000': 0
}

for length in lengths:
    if length < 1000:
        length_dict['0-999'] += 1
    elif 1000 <= length < 2000:
        length_dict['1000-1999'] += 1
    elif 2000 <= length < 3000:
        length_dict['2000-2999'] += 1
    elif 3000 <= length < 4000:
        length_dict['3000-3999'] += 1
    elif 4000 <= length < 5000:
        length_dict['4000-4999'] += 1
    elif 5000 <= length < 6000:
        length_dict['5000-5999'] += 1
    elif 6000 <= length < 7000:
        length_dict['6000-6999'] += 1
    elif 7000 <= length < 8000:
        length_dict['7000-7999'] += 1
    elif 8000 <= length < 9000:
        length_dict['8000-8999'] += 1
    else:
        length_dict['9000-10000'] += 1
    
length_dict

{'0-999': 1291,
 '1000-1999': 8709,
 '2000-2999': 0,
 '3000-3999': 0,
 '4000-4999': 0,
 '5000-5999': 0,
 '6000-6999': 0,
 '7000-7999': 0,
 '8000-8999': 0,
 '9000-10000': 0}

In [17]:
sum([1 for shelf_value in shelf_values if len(shelf_value) == 1000])

8709

In [35]:
sum([1 for shelf_value in shelf_values if
    any([
        any([
            synonym in word
        for synonym in ['fun', 'happy', 'cheer', 'joy']])
    for word in shelf])
])

10000

In [42]:
next(
    word for word in shelf_values[9999] if any([
        synonym in word for synonym in ['fun', 'happy', 'cheer', 'joy']
    ]
    )
)

'funny'

In [43]:
book_links[9999], shelf_values[9999]

('/book/show/35503.Without_Feathers',
 ['to-read',
  'humor',
  'currently-reading',
  'fiction',
  'short-stories',
  'comedy',
  'favorites',
  'humour',
  'own',
  'non-fiction',
  'owned',
  'essays',
  'american',
  'funny',
  'default',
  'books-i-own',
  'nonfiction',
  'contemporary',
  'literature',
  'owned-books',
  'wishlist',
  'woody-allen',
  'audiobook',
  'stories',
  'plays',
  'library',
  'short-story',
  '20th-century',
  '2010',
  'to-buy',
  'jewish',
  'american-literature',
  'short-fiction',
  'my-library',
  '2018',
  '2014',
  'humor-satire',
  'usa',
  'umorismo',
  'play',
  '2017',
  '2016',
  '2015',
  'collection',
  'audiobooks',
  '2012',
  'satire',
  '2019',
  'memoir',
  'narrativa',
  'my-books',
  '2011',
  'drama',
  '2020',
  'contos',
  'novels',
  'literatura',
  'english',
  '50-books-that-will-change-your-life',
  'united-states',
  'i-own',
  'contemporary-fiction',
  '1970s',
  'americana',
  'short-story-collection',
  'finished',
  'ant

In [166]:
'fun' in label_counts, label

(True, 4140)

In [168]:
label_counts['fun'], label_counts['happy'], label_counts['sad'], label_counts['tragic']

(4140, 88, 1714, 230)

### Possible Scraping of Comments